Instal sheXer using pip. It is compatible with Python 3.7, 3.8, 3.9, 3.10, 3.11

In [ ]:


!pip install shexer



To execute shexer you need to create an instance of the object Shaper. This object will be built with all the parameters that you need to tune the extraction process. You need make a mandatory decission of 2 things: which is your data source and what shapes you want to extract from it. Beyonf that, you can set many other parameters to modify how sheXer extract shapes, but all those parameters have default values.

In [3]:
from shexer.shaper import Shaper

In this case, we are getting a shape for the class Q7187 (gene). For that we are using 50 random instances of such class and we are consuming data from Wikidata's endpoint.

In [ ]:
shape_map_raw = 'SPARQL "select ?s where {  ?s <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q7187> .   } limit 50"@<ActiveSite>'

shaper = Shaper(shape_map_raw=shape_map_raw,
                url_endpoint="https://query.wikidata.org/sparql",
                instantiation_property="http://www.wikidata.org/prop/direct/P31")

str_result = shaper.shex_graph(string_output=True)
print(str_result)


The results obtained are pretty ugly. But we can, for example, bind some namespaces there to make it more readable. Also, we can tell shexer to annotate with a label every Wikidata elements that it recognizes.

In [ ]:
shape_map_raw = 'SPARQL "select ?s where {  ?s <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q7187> .   } limit 50"@<ActiveSite>'

namespaces_dict = {
            "http://example.org/": "ex",
            "http://www.w3.org/XML/1998/namespace/": "xml",
            "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
            "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
            "http://www.w3.org/2001/XMLSchema#": "xsd",
            "http://xmlns.com/foaf/0.1/": "foaf",
            "http://www.wikidata.org/prop/direct/" : "wdt",
            "http://www.wikidata.org/entity/" : "wd"
            }

shaper = Shaper(shape_map_raw=shape_map_raw,
                url_endpoint="https://query.wikidata.org/sparql",
                instantiation_property="http://www.wikidata.org/prop/direct/P31",
                namespaces_dict=namespaces_dict,
                wikidata_annotation=True)

str_result = shaper.shex_graph(string_output=True)
print(str_result)


We can also tell sheXer to ignore triples using properties of some namespaces. For example, this can be useful to skip non-direct triples. We could also maybe tell sheXer to skip any triple constraint which is not obsreved among at least half of the tracked instances.

In [ ]:
shape_map_raw = 'SPARQL "select ?s where {  ?s <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q7187> .   } limit 50"@<ActiveSite>'

namespaces_dict = {
            "http://example.org/": "ex",
            "http://www.w3.org/XML/1998/namespace/": "xml",
            "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
            "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
            "http://www.w3.org/2001/XMLSchema#": "xsd",
            "http://xmlns.com/foaf/0.1/": "foaf",
            "http://www.wikidata.org/prop/direct/" : "wdt",
            "http://www.wikidata.org/entity/" : "wd"
            }
namespaces_to_ignore = ["http://www.wikidata.org/prop/",
                        "http://www.w3.org/2004/02/skos/core#",
                        "http://schema.org/",
                        "http://wikiba.se/ontology#",
                        "http://www.wikidata.org/prop/direct-normalized/"]


shaper = Shaper(shape_map_raw=shape_map_raw,
                url_endpoint="https://query.wikidata.org/sparql",
                url_graph_input="https://github.com/kg-subsetting/simple_3000_dump/raw/main/random_slice.nt",
                instantiation_property="http://www.wikidata.org/prop/direct/P31",
                namespaces_dict=namespaces_dict,
                namespaces_to_ignore=namespaces_to_ignore,
                wikidata_annotation=True)

str_result = shaper.shex_graph(string_output=True,
                               acceptance_threshold=0.5)
print(str_result)


sheXer becomes faster when it does not depend on external endpoint, but parses RDF content. Ypu can parse it in several ways. In the next example we are feeding shexer with some NT file containing a partial wikidata (RDF) dump.
Instead of using shape maps, we can just tell shexer to get a shape for every element with at least an instance. 

In [ ]:
# shape_map_raw = 'SPARQL "select ?s where {  ?s <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q7187> .   } limit 50"@<ActiveSite>'

namespaces_dict = {
            "http://example.org/": "ex",
            "http://www.w3.org/XML/1998/namespace/": "xml",
            "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
            "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
            "http://www.w3.org/2001/XMLSchema#": "xsd",
            "http://xmlns.com/foaf/0.1/": "foaf",
            "http://www.wikidata.org/prop/direct/" : "wdt",
            "http://www.wikidata.org/entity/" : "wd"
            }
namespaces_to_ignore = ["http://www.wikidata.org/prop/",
                        "http://www.w3.org/2004/02/skos/core#",
                        "http://schema.org/",
                        "http://wikiba.se/ontology#",
                        "http://www.wikidata.org/prop/direct-normalized/"]


shaper = Shaper(# shape_map_raw=shape_map_raw,
                # url_endpoint="https://query.wikidata.org/sparql",
                url_graph_input="https://github.com/kg-subsetting/simple_3000_dump/raw/main/random_slice.nt",
                instantiation_property="http://www.wikidata.org/prop/direct/P31",
                all_classes_mode=True,
                namespaces_dict=namespaces_dict,
                namespaces_to_ignore=namespaces_to_ignore,
                wikidata_annotation=True)

str_result = shaper.shex_graph(string_output=True,
                               acceptance_threshold=0.5)
print(str_result)


Or we could be more specific and thell sheXer to get a shape for just one/several specific classes.

In [ ]:
# shape_map_raw = 'SPARQL "select ?s where {  ?s <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q7187> .   } limit 50"@<ActiveSite>'

namespaces_dict = {
            "http://example.org/": "ex",
            "http://www.w3.org/XML/1998/namespace/": "xml",
            "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
            "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
            "http://www.w3.org/2001/XMLSchema#": "xsd",
            "http://xmlns.com/foaf/0.1/": "foaf",
            "http://www.wikidata.org/prop/direct/" : "wdt",
            "http://www.wikidata.org/entity/" : "wd"
            }
namespaces_to_ignore = ["http://www.wikidata.org/prop/",
                        "http://www.w3.org/2004/02/skos/core#",
                        "http://schema.org/",
                        "http://wikiba.se/ontology#",
                        "http://www.wikidata.org/prop/direct-normalized/"]


shaper = Shaper(# shape_map_raw=shape_map_raw,
                # url_endpoint="https://query.wikidata.org/sparql",
                url_graph_input="https://github.com/kg-subsetting/simple_3000_dump/raw/main/random_slice.nt",
                instantiation_property="http://www.wikidata.org/prop/direct/P31",
                target_classes=["wd:Q11424"],
                namespaces_dict=namespaces_dict,
                namespaces_to_ignore=namespaces_to_ignore,
                wikidata_annotation=True)

str_result = shaper.shex_graph(string_output=True,
                               acceptance_threshold=0.5)
print(str_result)


There are many ways to provide input and to configure what shaopes you want to get or how are they supposed to look like. Check https://github.com/DaniFdezAlvarez/shexer to dig deeper into this tool. 